# Introduction to ClimaCore.jl's API concepts

## What is ClimaCore?

A suite of tools for constructing spatial discretizations

- primarily aimed at climate and weather models
- initial aim:
  - spectral element discretization in the horizontal
  - staggered finite difference in the vertical
  - support for Cartesian and spherical domains
- currently under development

In [ ]:
using Pkg
Pkg.add("ClimaCore")
Pkg.add("ClimaCorePlots")
Pkg.add("LinearAlgebra")
Pkg.add("IntervalSets")
Pkg.add("Plots")
Pkg.add("Statistics")

using ClimaCore: Domains, Meshes, Geometry, Topologies, Spaces, Fields, Operators
using  ClimaCorePlots, LinearAlgebra, IntervalSets, Plots, Statistics

## 1. Constructing a discretization

In ClimaCore's API, a spatial discretization is made up of 4 elements:
- `Domain`
- `Mesh`
- `Topology`
- `Space`

### 1.1 Domains
These can be: `IntervalDomain`, `RectangleDomain` or `SphereDomain`.

An example of an `IntervalDomain` with non-periodic boundaries (we need to specify
`boundary_names`):

In [ ]:
column_domain = Domains.IntervalDomain(
    Geometry.ZPoint(0.0) .. Geometry.ZPoint(10.0),
    boundary_names = (:bottom, :top),
)

An example of a `RectangleDomain`, as the cross product of two `IntervalDomain`s:

In [ ]:
rectangle_domain = Domains.RectangleDomain(
        Domains.IntervalDomain(
            Geometry.XPoint(0.0),
            Geometry.XPoint(10.0);
            periodic = false,
            boundary_names = (:west, :east),
        ),
        Domains.IntervalDomain(
            Geometry.YPoint(0.0),
            Geometry.YPoint(5.0);
            periodic = false,
            boundary_names = (:south, :north),
        )
    )

An example of a `SphereDomain`, i.e., a circle of a given radius that can be used to construct a cubed-sphere:

In [ ]:
sphere_domain = Domains.SphereDomain(1.0)

### 1.2 Meshes

A `Mesh` is a division of a domain into elements. Three types:

- `IntervalMesh`, for 1D rectilinear mesh.
- `RectilinearMesh`, for 2D rectilinear mesh.
- Cubed-sphere meshes.

In [ ]:
column_mesh = Meshes.IntervalMesh(column_domain, nelems = 10)

Supported keywords for `IntervalMesh`: `stretching` (default is `Uniform()`), and `nelems`.

A `RectilinearMesh` is a mesh made of equally-spaced quad elements. It can be constructed in two ways:
- `RectilinearMesh(domain::RectangleDomain, n1, n2)`, given a `RectangleDomain` and the number of elements to use in each direction
- `RectilinearMesh(intervalmesh1::IntervalMesh1, intervalmesh2::IntervalMesh2)`, as the tensor product of two `IntervalMesh`es.
Examples:

In [ ]:
rectangle_mesh = Meshes.RectilinearMesh(rectangle_domain, 10, 5)
tensor_prod_rectangle_mesh = Meshes.RectilinearMesh(column_mesh, column_mesh)

A cubed-sphere mesh requires a `SphereDomain` and the number of elements across each panel of the cube. Example:

In [ ]:
sphere_mesh = Meshes.EquiangularCubedSphere(Domains.SphereDomain(1.0), 3)

Currently we support three different cubed-sphere specifications: `EquiangularCubedSphere`, `EquidistantCubedSphere`, and `ConformalCubedSphere`.

### 1.3 Topologies

A `Topology` determines the ordering and connections between elements of a mesh. Examples of Topologies we can construct with the three different types of mesh seen so far:

In [ ]:
interval_topology = Topologies.IntervalTopology(column_mesh)

In [ ]:
rectangle_topology = Topologies.Topology2D(rectangle_mesh)

In [ ]:
sphere_topology = Topologies.Topology2D(sphere_mesh)

### 1.4 Spaces

A `Space` represents a discretized function space over some domain.
Currently two main discretizations are supported: Spectral Element Discretization (both Continuous Galerkin and Discontinuous Galerkin types), a staggered Finite Difference Discretization, and the combination of these two in what we call a _hybrid_ space.

#### 1.4.1 Staggered finite difference discretization

This discretizes an interval domain by approximating the function by a value at either the center of each element (`CenterFiniteDifferenceSpace`), or the interfaces (points in 1D) between elements (`FaceFiniteDifferenceSpace`).

You can construct either the center or face space from the mesh, then construct the opposite space from the original one (this is to avoid allocating additional memory). 

In [ ]:
column_center_space = Spaces.CenterFiniteDifferenceSpace(column_mesh)
# construct the face space from the center one (or vice versa)
column_face_space = Spaces.FaceFiniteDifferenceSpace(column_center_space)

#### 1.4.2 Spectral element discretization

In finite element formulations, the weak form of a Partial Differential Equation (PDE)---which involves integrating all terms in the PDE over the domain---is evaluated on a subdomain $\Omega_e$ (element) and the local results are composed into a larger system of equations that models the entire problem on the global domain $\Omega$.

A spectral element space is a function space in which each function is approximated with a finite-dimensional polynomial interpolation in each element. Hence, we use polynomials as basis functions to approximate a given function (e.g., solution state). There are different ways of defininig basis functions: _nodal_ basis functions and _modal_ basis functions. We use _nodal_ basis functions (e.g. by using Lagrange interpolation), which are defined via the values of the polynomials at particular nodal points in each element (termed Finite Element *nodes*). Even though the basis functions can interpolate globally, it’s better to limit each function to interpolate locally within each element, so to avoid a dense matrix system of equations when adding up the element contributions on the global domain $\Omega$.

The Finite Element nodes can be chosen to coincide with those of a particular *quadrature rule*, (this is referred to as using _collocated_ nodes) which allows us to integrate functions over the domain. 

Let us give a concrete example of strong and weak form of a PDE. A Poisson's problem (in strong form) is given by

$$
   \nabla \cdot \nabla u = f, \textrm{ for  } \mathbf{x} \in \Omega .
$$

To obtain the weak form, let us multiply all terms by a test function $v$ and integrate by parts (i.e., apply the divergence theorem in multiple dimensions):

$$
   \int_\Omega \nabla v \cdot \nabla u \, dV - \int_{\partial \Omega} v \nabla u \cdot \hat{\mathbf n}\, dS = \int_\Omega  v f \, dV .
$$

Often, we choose to represent a field (say, the velocity field) such that $\nabla u \cdot \hat{\mathbf n} = 0$, so that we're only left with the volumetric parts of the equation above. 

The only supported choice for now is a `Gauss-Legendre-Lobatto` rule and nodes.

In [ ]:
# Gauss-Legendre-Lobatto quadrature with 4 nodes in each direction (16 in each 2D element)
quad = Spaces.Quadratures.GLL{4}()
# For a 1D spectral element space:
column_space = Spaces.SpectralElementSpace1D(interval_topology, quad)
# For a 2D Cartesian spectral element space:
rectangle_space = Spaces.SpectralElementSpace2D(rectangle_topology, quad)
# For a 2D spherical surface:
sphere_space = Spaces.SpectralElementSpace2D(sphere_topology, quad)

To summarize, the four different objects needed to set up a discretization in ClimaCore, are:

<img src="APIobjects.png" alt="ClimaCore API objects" width="400"/>

Once we define a `Space`, we can define a `Field` on it.

#### 1.4.3 Hybrid spaces
Let us start with defining a 2D hybrid space: XZ plane, comprised of 1D CG x 1D FD discretizations:


In [ ]:
# Set up a horizontal 1D space in the X-direction
horzdomain_1D = Domains.IntervalDomain(
    Geometry.XPoint{Float64}(0),
    Geometry.XPoint{Float64}(10),
    periodic = true,
)
horzmesh_1D = Meshes.IntervalMesh(horzdomain_1D; nelems = 10)
horztopology_1D = Topologies.IntervalTopology(horzmesh_1D)
horzspace_1D = Spaces.SpectralElementSpace1D(horztopology_1D, quad)

# Now let's build the hybrid 1DX (XZ) center space
hv_center_space_1DX = Spaces.ExtrudedFiniteDifferenceSpace(horzspace_1D, column_center_space)
# And corresponding hybrid 1DX (XZ) face space
hv_face_space_1DX = Spaces.FaceExtrudedFiniteDifferenceSpace(hv_center_space_1DX)

Next, we can define a 3D hybrid space: XYZ space, comprised of 2D CG x 1D FD discretizations:

In [ ]:
# Let us reuse the SpectralElement2D rectangle space we defined previously as the horizontal space now.
# Let's build the hybrid 2DX (XYZ) center space
hv_center_space_2DX = Spaces.ExtrudedFiniteDifferenceSpace(rectangle_space, column_center_space) 
# And corresponding hybrid 2DX (XYZ) face space
hv_face_space_2DX = Spaces.FaceExtrudedFiniteDifferenceSpace(hv_center_space_2DX)

Sketch of a 2DX hybrid discretization:

<img src="DiscretizationSketch.png" alt="3D hybrid discretization in a Cartesian domain" width="400"/>

In [ ]:
# Spherical extruded space center space
hv_sphere_center_space_2DX = Spaces.ExtrudedFiniteDifferenceSpace(sphere_space, column_center_space)
# And corresponding face space
hv_sphere_face_space_2DX = Spaces.FaceExtrudedFiniteDifferenceSpace(hv_sphere_center_space_2DX)

Sketch of an extruded 3D spherical shell hybrid discretization:

<img src="ExtrudedSphere.png" alt="3D hybrid discretization in a spherical shell" width="400"/>

## 2. Fields

Finally, on a given `Space`, we can construct a `Field`, which can represent mathematically either a scalar-valued field, a vector-valued field, or a combination of these two. A field is simply the association of a space and the values at each node in the space.

The easiest field to construct is the _coordinate field_

In [ ]:
coord = Fields.coordinate_field(rectangle_space)

This is a *struct-value field*: it contains coordinates in a struct at each point.
We can extract just the `x` coordinate, to get a *scalar field*:

In [ ]:
x = coord.x

Although you can't index directly into a field, it can be used in some other ways
similar to a Julia `Array`. For example, broadcasting can be used to define new fields in terms of other ones:

In [ ]:
sinx = sin.(x)

This works similarly for finite difference discretizations

In [ ]:
column_center_coords = Fields.coordinate_field(column_center_space)
column_face_coords = Fields.coordinate_field(column_face_space)

In [ ]:
plot(sin.(column_center_coords.z), ylim = (0.0, 10.0))
plot!(cos.(column_face_coords.z), ylim = (0.0, 10.0))

Reduction operations are defined anologously:

- `sum` will give the approximate integral of `v` or `f.(v)` over the domain $\Omega$. In an `AbstractSpectralElementSpace`, an integral over the entire space is computed by summation over the elements of the integrand multiplied by the Jacobian determinants and the quadrature weights at each node within an element. Hence, `sum` is computed by summation of the field values multiplied by the Jacobian determinants and quadrature weights:
$$
\sum_i f(v_i) W_i J_i \approx \int_\Omega f(v) \, d \Omega
$$
where $v_i$ is the value at each node, and $f$ is the identity function if not specified. This analogously translates to `AbstractFiniteDifferenceSpace`s.

Example:

In [ ]:
sum(sinx) ## integral

- `mean` gives the mean value of `field` or `f.(field)` over the domain, weighted by area. Similar to `sum`, in an `AbstractSpectralElementSpace`, this is computed by summation of the field values multiplied by the Jacobian determinants and quadrature weights
$$
\frac{\sum_i f(v_i) W_i J_i}{\sum_i W_i J_i} \approx \frac{\int_\Omega f(v) \, d \Omega}{\int_\Omega \, d \Omega}
$$
where $v_i$ is the Field value at each node, and $f$ is the identity function if not specified. This analogously translates to `AbstractFiniteDifferenceSpace`s.

Example:

In [ ]:
# mean(sinx) ## mean

- `norm`: 
The approximate $L^p$ norm of $v$, where $L^p$ represents the space of measurable
functions for which the p-th power of the absolute value is Lebesgue integrable, that is:

$$
\| v \|_p = \left( \int_\Omega |v|^p d \Omega \right)^{1/p}
$$

where $|v| $ is defined to be the absolute value if $v$ is a scalar-valued Field, or the 2-norm if it is a vector-valued Field or composite Field (see [LinearAlgebra.norm](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm)).
Similar to `sum` and `mean`, in an `AbstractSpectralElementSpace`, this is computed by summation of the field values multiplied by the Jacobian determinants and quadrature weights.
If `normalize=true` (the default), then internally the discrete norm is divided by the sum of the Jacobian determinants and quadrature weights:
$$
\left(\frac{\sum_i |v_i|^p W_i J_i}{\sum_i W_i J_i}\right)^{1/p} \approx \left(\frac{\int_\Omega |v|^p \, d \Omega}{\int_\Omega \, d \Omega}\right)^{1/p}
$$
If `p=Inf`, then the norm is the maximum of the absolute values
$$
\max_i |v_i| \approx \sup_{\Omega} |v|
$$

Consequently all norms should have the same units for all $p$ (being the same as calling `norm` on a single value).

If `normalize=false`, then the denominator term is omitted, and so the result will be the norm as described above multiplied by the length/area/volume of $\Omega$ to the power of $1/p$. Similar to `sum` and `mean`, this analogously translates to `AbstractFiniteDifferenceSpace`s.

Example:

In [ ]:
norm(sinx) ## L² norm

Another very simple field we can think of is a field that has all zero entries:


In [ ]:
zero_field = zeros(sphere_space)

Or a field that has all entries equal to 1:

In [ ]:
one_field = ones(sphere_space)

With this, we can easily compute the surface area or volume of a given domain $\Omega$. For example:

In [ ]:
sphere_surface = sum(one_field)

## 3. Points, vectors and vector fields



### 3.1 Points
Points represent _locations_ in space (not to be confused with _positions_, _position vectors_ or _location vectors_ in geometry). Vectors, on the other hand, represent displacements in space. An analogy with time works well: times, (also called instants or datetimes) are _locations_ in time. While, durations are displacements in time.

ClimaCore supports the following types of points in 1D, 2D, and 3D, in Cartesian or spherical geometries:
- `XPoint(x)`
- `YPoint(y)`
- `ZPoint(z)`
- `XYPoint(x, y)`
- `XZPoint(x, z)`
- `XYZPoint(x, y, z)`
- `LatLongPoint(lat, long)`
- `LatLongZPoint(lat, long, z)`

Examples of points in Cartesian geoemetries:

In [ ]:
# Cartesian:
# A point in 1D, along the X-axis:
CP1 = Geometry.XPoint(0.0)
# A point on the 2D XY plane (which lies on the X-axis):
CP2 = Geometry.XYPoint(1.0,0.0)
# A point in 3D, in the XYZ space:
CP3 = Geometry.XYZPoint(1.0, 2.0, 3.0)

Examples of points on spherical geometries:

In [ ]:
# Spherical:
# A point on the 2D sphere surface:
SP1 = Geometry.LatLongPoint(0.0, -180.0)
# A point in the 3D extruded sphere, where Z is the elevation
SP2 = Geometry.LatLongZPoint(0.0, 180.0, 1000.0)

**Note 1**: In ClimaCore we use angles (and, therefore, trigonometric functions: `cosd`, `sind`, `acosd`, `asind`, `tand`,...) in degrees, not in radians! Moreover, `lat` (usually denoted by $\theta$) $\in [-90.0, 90.0]$, and `long` (usually denoted by $\lambda$) $\in [-180.0, 180.0]$.

**Note 2:**: In a `Geometry.LatLongZPoint(lat, long, z)`, `z` represents the elevation above the surface of the sphere with radius R (implicitly accounted for in the geoemtry).

**Note 3**: There are also a set of specific Cartesian points (`Cartesian1Point(x1)`, `Cartesian2Point(x2)`, etc). These are occasionally useful for converting everything to a full Cartesian domain (e.g. for visualization purposes). These are distinct from `XYZPoint` as `ZPoint` can mean different things in different domains.

### 3.2 Vectors and vector fields

_Vector_ can mean a few things depending on context:

- In Julia, a `Vector` is just an ordered collection of values (i.e., a container).
- In mathematics, a vector is an element of a _vector space_: a set of objects, which may be added together and multiplied by a scalar.
- In physics, a vector typically refers to a directional quantity: that is something with both a _direction_ and a _magnitude_. This is the best way to think of vectors in ClimaCore.

A _vector field_ is then a vector-valued field: that is an assignment of a vector to each point in a space. For instance, a vector field in the plane can be visualised as a collection of arrows with a given magnitude and direction, each attached to a point in the plane. 

In a coordinate system, a vector field on a domain in n-dimensional Euclidean space can be represented as a vector-valued function that associates an n-tuple of real numbers to each point of the domain. This representation of a vector field _depends on the coordinate system_, and there are transformation laws for passing from one coordinate system to the other. 

ClmaCore supports different coordinate systems and, therefore, vector representations. In fact, one of the key requirements of ClimaCore is to support vectors specified in orthogonal (Cartesian) and curvilinear coordinate systems. 

#### 3.2.1 `LocalVector`: `UVector`, `UVVector`, and `UVWVector`, etc; a "universal" basis

The easiest basis to use is the "UVW" basis, which can be defined in both Cartesian or spherical domains:
- in a Cartesian domain, it is the usual Cartesian orthogonal vector basis (U along the X-axis, V along the Y-axis, W along the Z-axis).
- in a spherical domain, it is the orthogonal basis relative to spherical (curvilinear) coordinates:
  - U is the zonal (eastward) component
  - V is the meridonal (northward) component
  - W is the radial component

It has some nice properties which make it convenient:
 - it's an orthonormal basis:
   - it is easy to decompose a vector (take the projection along the basis)
   - the components are easy to interpret (they have unit scale)
 - allow us to write code across domains
   - U and V are always horizontal, W is vertical

We can define "generic" vectors via `UVector`, `UVVector`, and `UVWVector` that can be equally defined on Cartesian or spherical spaces.

Examples:

In [ ]:
# Generic vectors in 1D
u₁ = Geometry.UVector(1.0)
u₂ = Geometry.VVector(1.0)
u₃ = Geometry.WVector(1.0)

# Generic vectors in 2D
uₕ₁₂ = Geometry.UVVector(1.0, 1.0)
uₕ₁₃ = Geometry.UWVector(1.0, 1.0)
uₕ₂₃ = Geometry.VWVector(1.0, 1.0)

# Generic vector in 3D
u = Geometry.UVWVector(1.0, 1.0, 1.0)

But if we need to compute with them, or feed differential operators with them, then may want to consider different bases, as not all operators accept all bases.

#### 3.2.2 Covariant and Contravariant bases

<img src="Bases.png" alt="ifferent bases supported in ClimaCore" width="400"/>

_Covariance_ and _contravariance_ describe how the quantitative description of certain geometric or physical entities changes with a change of basis. Quoted from the [Covariance and Contravariance Wikipedia page](https://en.wikipedia.org/wiki/Covariance_and_contravariance_of_vectors):

> In physics, a basis is sometimes thought of as a set of reference axes. A change of scale on the reference axes corresponds to a change of units in the problem. For instance, by changing scale from meters to centimeters (that is, _dividing_ the scale of the reference axes by 100), the components of a measured velocity vector are _multiplied_ by 100. Vectors exhibit this behavior of changing scale _inversely_ to changes in scale to the reference axes and consequently are called **contravariant**. As a result, vectors often have units of distance or distance with other units (as, for example, velocity has units of distance divided by time).

> In contrast, covectors (also called _dual_ vectors) typically have units of the inverse of distance or the inverse of distance with other units. An example of a covector is the gradient, which has units of a spatial derivative, or (distance)$^{-1}$. The components of covectors change in the _same way_ as changes to scale of the reference axes and consequently are called **covariant**.

> Under more general changes in basis:
> - A _contravariant vector_ or tangent vector (often abbreviated simply as _vector_, such as a direction vector or velocity vector) has components that _contra-vary_ with a change of basis to compensate. That is, the matrix that transforms the vector components must be the inverse of the matrix that transforms the basis vectors. The components of vectors (as opposed to those of covectors) are said to be **contravariant**. Examples of vectors with _contravariant components_ include the position of an object relative to an observer, or any derivative of position with respect to time, including velocity, acceleration, and jerk. In Einstein notation, contravariant components are denoted with _upper indices_ as in
$$
{\displaystyle \mathbf {w} =w^{i}\mathbf {e} _{i}} \textrm{   (note: implicit summation over index "i")}
$$

> - A _covariant vector_ or cotangent vector (often abbreviated as _covector_) has components that _co-vary_ with a change of basis. That is, the components must be transformed by the same matrix as the change of basis matrix. The components of covectors (as opposed to those of vectors) are said to be **covariant**. Examples of covariant vectors generally appear when taking a gradient of a function. In Einstein notation, covariant components are denoted with _lower indices_ as in
$$
{\displaystyle \mathbf {v} = v_{i}\mathbf {e} ^{i}.}
$$

In our case, the _covariant basis_ is specified by the partial derivative of the transformation from the reference element $\xi \in [-1,1]^d$ to $x$ in our physical space:
$$
\mathbf{e}_i = \frac{\partial x}{\partial \xi^i}
$$
while the _contravariant basis_ is the opposite: gradient in $x$ of the coordinate (the inverse map)
$$
\mathbf{e}^i = \nabla_x \xi^i
$$


Examples:

In [ ]:
Cov1 = Geometry.Covariant1Vector(1.0) # a vector expressed in terms of its 1st covariant component 
Cov2 = Geometry.Covariant2Vector(1.0) # a vector expressed in terms of its 2nd covariant component
Cov3 = Geometry.Covariant3Vector(1.0) # a vector expressed in terms of its 3rd covariant component

Cov12 = Geometry.Covariant12Vector(1.0, 1.0) # a vector expressed in terms of its 1st & 2nd covariant components
Cov13 = Geometry.Covariant13Vector(1.0, 1.0) # a vector expressed in terms of its 1st & 3rd covariant components
Cov23 = Geometry.Covariant23Vector(1.0, 1.0) # a vector expressed in terms of its 2nd & 3rd covariant components

Cov123 = Geometry.Covariant123Vector(1.0, 1.0, 1.0) # a vector expressed in terms of its 1st, 2nd & 3rd covariant components

Con1 = Geometry.Contravariant1Vector(1.0) # a vector expressed in terms of its 1st contravariant component
Con2 = Geometry.Contravariant2Vector(1.0) # a vector expressed in terms of its 2nd contravariant component
Con3 = Geometry.Contravariant3Vector(1.0) # a vector expressed in terms of its 3rd contravariant component

Con12 = Geometry.Contravariant12Vector(1.0, 1.0) # a vector expressed in terms of its 1st & 2nd contravariant components
Con13 = Geometry.Contravariant13Vector(1.0, 1.0) # a vector expressed in terms of its 1st & 3rd contravariant components
Con23 = Geometry.Contravariant23Vector(1.0, 1.0) # a vector expressed in terms of its 2nd & 3rd contravariant components

Con123 = Geometry.Contravariant123Vector(1.0, 1.0, 1.0) # a vector expressed in terms of its 1st, 2nd & 3rd contravariant components

**Note**:

* these are specific to a given element: you generally can't compare covariant or contravariant component from one element with that in another
  - in this case, you need to first convert them to UVW basis (e.g. we do this for DSS operations)

* we choose the coordinates of the reference element so that $\xi^1$ and $\xi^2$ are horizontal, and $\xi^3$ is vertical
  - in a Cartesian domain, this means that covariant and contravariant components are just rescaled versions of the UVW components

* things get a little more complicated in the presence of terrain, but $\xi^3$ is radially aligned 
  - the 3rd covariant component is aligned with W, but the 3rd contravariant component may not be (e.g. at the surface it is normal to the boundary)


#### 3.2.3 Cartesian bases
Analogously to `CartesianPoint`s, in ClimaCore, there are also `CartesianVector`s: these allow conversion to a global Cartesian basis. It is intended mainly for visualization purposes.

# 3.3 Conversions

To convert between different vector bases, you need a `LocalGeometry` object: this contains all the necessary information (coordinates, metric terms, etc) to do the conversion. These are constructed as part of the `Space`.

In [ ]:
coords = Fields.coordinate_field(rectangle_space)

# option 1: use the "LocalGeometry" field to define a vector
function f(local_geometry)
    coord = local_geometry.coordinates
    u = Geometry.UVVector(sin(coord.x), cos(coord.y)) # define u in a UV basis
    return Geometry.CovariantVector(u, local_geometry) # convert u to a covariant basis
end
# Fields.local_geometry_field(space) returns a Field of "LocalGeometry" objects
uᵢ = map(f, Fields.local_geometry_field(rectangle_space))

# option 2: when broadcasting vector constructors, we silently insert the LocalGeometry as an extra argument
# e.g. CovariantVector.(u) is really doing CovariantVector.(u, Fields.local_geometry_field(axes(u)))
coords = Fields.coordinate_field(rectangle_space)
uᵢ = Geometry.CovariantVector.(Geometry.UVVector.(sin.(coords.x), cos.(coords.y))) # define a vector field in the UV basis, and convert it to a CovariantVector (casting)


# 4. Operators

_Operators_ can compute spatial derivative operations.

 - for performance reasons, we need to be able to "fuse" multiple operators and function applications
 - Julia provides a tool for this: **broadcasting**, with a very flexible API

Can think of operators are "pseudo-functions": can't be called directly, but act similar to functions in the context of broadcasting. They are matrix-free, in the sense that we define the _action_ of the operator directly on a field, without explicitly assembling the matrix representing the discretized operator.

## 4.1 Spectral element operators

The `Gradient` operator takes the gradient of a scalar field, and returns a vector field.

In [ ]:
grad = Operators.Gradient()
∇sinx = grad.(sinx)

In [ ]:
plot(∇sinx.components.data.:1, clim = (-1, 1))

This returns the gradient in _covariant_ coordinates
$$
(\nabla f)_i = \frac{\partial f}{\partial \xi^i}
$$
where $(\xi^1,\xi^2)$ are the coordinates in the *reference element*: a square $[-1,1]^2$.

This can be converted to a local orthogonal basis by multiplying by the partial derivative matrix
$$
\frac{\partial \boldsymbol{\xi}}{\partial \boldsymbol{x}}
$$
This can be done calling `ClimaCore.Geometry.LocalVector`:

In [ ]:
∇sinx_cart = Geometry.LocalVector.(∇sinx)

In [ ]:
plot(∇sinx_cart.components.data.:1, clim = (-1, 1))

In [ ]:
plot(∇sinx_cart.components.data.:2, clim = (-1, 1))

In [ ]:
∇sinx_ref = Geometry.UVVector.(cos.(x), 0.0)
norm(∇sinx_cart .- ∇sinx_ref)

Similarly, the `Divergence` operator takes the divergence of vector field, and returns a scalar field.

If we take the divergence of a gradient, we can get a Laplacian:

In [ ]:
div = Operators.Divergence()
∇²sinx = div.(grad.(sinx))
plot(∇²sinx)

**Note**: In curvilinear coordinates, the divergence is defined in terms of the _contravariant_ components $u^i$:
$$
\nabla \cdot u = \frac{1}{J} \sum_i \frac{\partial}{\partial \xi^i} (J u^i)
$$
The `Divergence` operator handles this conversion internally.

## 4.2 Finite difference operators

Finite difference operators are similar with some subtle differences:
- they can change staggering (center to face, or vice versa)
- they can span multiple elements
  - no DSS is required
  - boundary handling may be required

We use the following convention:
 - centers are indexed by integers `1, 2, ..., n`
 - faces are indexed by half integers `half, 1+half, ..., n+half`

**Face to center gradient**

A finite-difference operator defines a _stencil_, i.e., the set of cells or cell faces it reaches for its action at a given location $i$. For example, central difference the gradient operator is usually written as:

$$
\nabla\theta[i] = \frac{\theta [i+\tfrac{1}{2}] - \theta[i-\tfrac{1}{2}]}{\Delta z}
$$

However a gradient should return a vector, not a scalar! So it returns a `Contravariant3Vector` where

$$
(\nabla\theta)_3 [i] = \theta [i+\tfrac{1}{2}] - \theta[i-\tfrac{1}{2}]
$$


You might wonder where did the $1 / \Delta z$ go? Well, in our case it reappears when we convert from a covariant basis to a W basis, since
$$\Delta z = \frac{\partial z}{\partial \xi^3}$$


In [ ]:
cosz = cos.(column_face_coords.z)
gradf2c = Operators.GradientF2C()
∇cosz = gradf2c.(cosz)
Geometry.WVector.(∇cosz)

In [ ]:
plot(map(x -> x.w, Geometry.WVector.(∇cosz)), ylim = (0, 10))

**Center to face gradient**

Uses the same stencil, but doesn't work directly:

In [ ]:
sinz = sin.(column_center_coords.z)
gradc2f = Operators.GradientC2F()
# ∇sinz = gradc2f.(sinz) ## this would throw an error

This throws an error because face values at the boundary are _not_ well-defined:

```
...
      \
        ∇θ[2+½]
      /
θ[2]
      \
        ∇θ[1+½]
      /
θ[1]
      \
        ????
```

To handle boundaries we need to *modify the stencil*. Two options:
- provide the _value_ $\theta^*$ of $\theta$ at the boundary:
$$
\nabla\theta[\tfrac{1}{2}] = \frac{\theta[1] - \theta^*}{\Delta z /2}
$$

- provide the *gradient* $\nabla\theta^*$ of $\theta$ at the boundary:
$$
\nabla\theta[\tfrac{1}{2}] = \nabla\theta^*
$$

These modified stencils are provided as keyword arguments to the operator (based on the boundary label names):

In [ ]:
sinz = sin.(column_center_coords.z)
gradc2f = Operators.GradientC2F(
    bottom = Operators.SetValue(sin(0.0)),
    top = Operators.SetGradient(
        Geometry.WVector(cos(10.0)),
    ),
)
∇sinz = gradc2f.(sinz)

In [ ]:
plot(map(x -> x.w, Geometry.WVector.(∇sinz)), ylim = (0, 10))

As before, multiple operators (or functions) can be fused together with broadcasting.

One extra advantage of this is that boundaries of the inner operators only need to be specified if they would affect the final result.

Consider the center-to-center Laplacian:

```
...
      \       /
        ∇θ[2+½]
      /       \
θ[2]            ∇⋅∇θ[2]
      \       /
        ∇θ[1+½]
      /       \
θ[1]            ∇⋅∇θ[1]
              /
         ∇θ*
```

In [ ]:
sinz = sin.(column_center_coords.z)
# we don't need to specify boundaries, as the stencil won't reach that far
gradc2f = Operators.GradientC2F()
divf2c = Operators.DivergenceF2C(
    bottom = Operators.SetValue(Geometry.WVector(cos(0.0))),
    top = Operators.SetValue(Geometry.WVector(cos(10.0))),
)
∇∇sinz = divf2c.(gradc2f.(sinz))

In [ ]:
plot(∇∇sinz, ylim = (0, 10))

# 5 Exercises:

1. Spot a function or operator that is missing unit tests and try to add them.
2. Spot a function or operator that is missing documentation and try to add it.
3. If you are artistically inclined, you can try adding any visual aid to the docs.
4. Construct a rotational vector field around an arbitrary axis, and plot the u and v components.
5. "Extra credit/Bonus points": Challenge your ClimaCore expertise reproducing any of the examples on the [Chebfun sphere examples page](https://www.chebfun.org/examples/sphere/).